In [7]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.metrics import classification_report
import numpy as np

In [ ]:
model_path = '<OUR JAVANESE DISTILLBERT MODEL PATH>'

In [ ]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [ ]:
file_data_path = '<CORPUS FILE PATH>'

In [ ]:
# Load the data
data = pd.read_csv(file_data_path, encoding='ISO-8859-1')

In [ ]:
data

,sentence,label
0,basmat minangka karakter ing kitab purwaning d...,0
1,pedemoleque minangka permen tradisional saka b...,0
2,pencon yaiku sarupaning pntholanplnthukan kang...,0
3,colin the caterpillar yaiku salah sawijining j...,0
4,shaken not stirred minangka tembung rangking a...,0
...,...,...
37330,zuppa toscana minangka sawijining olahan istil...,0
37331,zutphen iku gemeente ing provinsi gelderland w...,0
37332,zwetschgenkuchen pflaumenkuchen utawa zwetschg...,0
37333,zwiebelkuchen jerman kanggo trikatuka cake iku...,0


In [ ]:
training_args = TrainingArguments(
       output_dir="<OUTPUT PATH",
       report_to="none"  # Disable reporting to wandb
   )

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
)


<ipython-input-14-3639832fc760>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
def preprocess_function(batch, tokenizer):
    texts = [text if isinstance(text, str) else "" for text in batch["sentence"]]

    tokenized = tokenizer(texts, truncation=True, padding=True)

    return tokenized


In [16]:
 # Create a dataset from the pandas dataframe
dataset = Dataset.from_pandas(data)

In [ ]:
# Tokenize the news dataset
tokenized_data = dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True)

Map:   0%|          | 0/37335 [00:00<?, ? examples/s]

In [18]:
# Initialize the Trainer
trainer = Trainer(model=model, tokenizer=tokenizer)

<ipython-input-18-03a48bd5bdcf>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, tokenizer=tokenizer)


In [ ]:
# Make predictions on the tokenized unseen dataset
dataset = trainer.predict(tokenized_data)
y_pred = dataset.predictions.argmax(-1)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: adryan-kusumawardhana (adryan-kusumawardhana-monash-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
y_true = data['label']

In [ ]:
data['predictions'] = y_pred

In [ ]:
from tqdm import tqdm

if "predictions" in data.columns and "sentence" in data.columns:
    if len(y_pred) < len(data):
        y_pred = np.pad(y_pred, (0, len(data) - len(y_pred)), constant_values=-1)

    data = data[data["predictions"] >= 0]

    label_prediction = "predictions"
    sentence_column = "sentence"

    output_df = data[[sentence_column, label_prediction]]

    print("\nProcessing rows:")
    for index, row in tqdm(data.iterrows(), total=len(data), unit="row"):
        sentence = row[sentence_column]
        predictions = row[label_prediction]

    prediction_counts = data[label_prediction].value_counts().clip(lower=0)
    total_samples = max(len(data), 1)
    prediction_percentages = ((prediction_counts / total_samples) * 100).round(2)

    sorted_by_category = prediction_percentages.sort_index()

    print("\nProportions for the predicted levels of Honorifics:")
    for category, percentage in sorted_by_category.items():
        print(f"Honorific Level: {category} | Percentage: {percentage}%")

else:
    print(f"Error: Ensure the DataFrame contains right columns.")



Processing rows:


100%|██████████| 37335/37335 [00:01<00:00, 25128.54row/s]


📊 Proportions for the predicted levels of Honorifics:
Honorific Level: 0 | Percentage: 89.2%
Honorific Level: 1 | Percentage: 2.95%
Honorific Level: 2 | Percentage: 6.23%
Honorific Level: 3 | Percentage: 1.62%
